In [6]:
import pandas as pd

In [14]:
pd.read_csv("./data/train.csv", encoding="utf-8")

,id,label,value
0,0,0,keyone=R89y0wdIlTHA2zgyd1CC&keytwo=jQuery21405...
1,1,0,keyone=7971532921&keytwo=7973850305&keythree=e...
2,2,1,keyone=jsonp807&keytwo=foo%2527%2529%2529%2529...
3,3,0,keyone=%25E9%25AD%2594%25E5%258A%259B%25E6%259...
4,4,0,keyone=1477491716660_1713&keytwo=1477534391431...
5,5,0,keyone=1477730577831_260&keytwo=7983342629
6,6,0,keyone=1477199864802_234&keytwo=12036&keythree...
7,7,0,keyone=1477138255498_51&keytwo=1478681578350&k...
8,8,0,keyone=4&keytwo=1030631513
9,9,0,keyone=7980372516&keytwo=7974180019&keythree=1...


In [17]:
pd.read_csv("./data/test.csv", encoding="utf-8")

,id,value
0,0,keyone=user%2520denied%2520your%2520request.&k...
1,1,keyone=1479202611276_599&keytwo=w2uHh951q
2,2,keyone=4&keytwo=7983990895&keythree=7740726305
3,3,keyone=data&keytwo=jsonp348&keythree=1119720249
4,4,keyone=20880674&keytwo=7977104874&keythree=788...
5,5,keyone=408749530&keytwo=it+won%27t+harm+anyone...
6,6,keyone=1478879528096&keytwo=1478607765366_581&...
7,7,keyone=7977048131&keytwo=1477365183431_193
8,8,keyone=477764104&keytwo=5&keythree=14779086901...
9,9,keyone=11331&keytwo=jsonp432&keythree=74023666...


In [10]:
# pd.read_csv("./data/test2.csv", encoding="utf-8")